In [16]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as f
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [22]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 10
epochs = 5
learning_rate = 0.005

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])


train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'sheep', 'truck')

In [12]:
class ConvModel(nn.Module):

    def __init__(self):
        super(ConvModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6 ,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(f.relu(self.conv1(x)))
        x = self.pool(f.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = f.relu(self.fc1(x))
        x = f.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [14]:
model = ConvModel().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [23]:
total_steps = len(train_loader)
step = round(0.1 * total_steps)

for epoch in range(epochs):

    for i, (images, labels) in enumerate(train_loader):

        images = images.to(device)
        labels = labels.to(device)

        prediction = model(images)

        loss = criterion(prediction, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % step == 0:
            print(f"epoch:{epoch+1}/{epochs}, step:{i+1}/{total_steps}, loss:{loss.item():.4f}")

print("finish training")

epoch:1/5, step:50/500, loss:2.2947
epoch:1/5, step:100/500, loss:2.3004
epoch:1/5, step:150/500, loss:2.2968
epoch:1/5, step:200/500, loss:2.3030
epoch:1/5, step:250/500, loss:2.3037
epoch:1/5, step:300/500, loss:2.3019
epoch:1/5, step:350/500, loss:2.2981
epoch:1/5, step:400/500, loss:2.3001
epoch:1/5, step:450/500, loss:2.3061
epoch:1/5, step:500/500, loss:2.2970
epoch:2/5, step:50/500, loss:2.3010
epoch:2/5, step:100/500, loss:2.2932
epoch:2/5, step:150/500, loss:2.2921
epoch:2/5, step:200/500, loss:2.3003
epoch:2/5, step:250/500, loss:2.2943
epoch:2/5, step:300/500, loss:2.2977
epoch:2/5, step:350/500, loss:2.3007
epoch:2/5, step:400/500, loss:2.2961
epoch:2/5, step:450/500, loss:2.3034
epoch:2/5, step:500/500, loss:2.3008
epoch:3/5, step:50/500, loss:2.3008
epoch:3/5, step:100/500, loss:2.3096
epoch:3/5, step:150/500, loss:2.3039
epoch:3/5, step:200/500, loss:2.2976
epoch:3/5, step:250/500, loss:2.3008
epoch:3/5, step:300/500, loss:2.2957
epoch:3/5, step:350/500, loss:2.3028
epoc

In [24]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_corrects = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:

        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)

        _, prediction = torch.max(outputs, 1)

        n_samples += labels.size(0)
        n_correct += (prediction == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = prediction[i]
            if label == pred:
                n_class_corrects[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples

    print(f"accuracy = {acc}")

    for i in range(10):
        acc = 100.0 * n_class_corrects[i] / n_class_samples[i]
        print(f" accuracy of {classes[i]}: {acc}%")




accuracy = 13.328
 accuracy of plane: 14.112903225806452%
 accuracy of car: 96.84418145956607%
 accuracy of bird: 0.0%
 accuracy of cat: 24.46808510638298%
 accuracy of deer: 0.0%
 accuracy of dog: 0.0%
 accuracy of frog: 0.1949317738791423%
 accuracy of horse: 0.0%
 accuracy of sheep: 0.0%
 accuracy of truck: 0.0%
